In [42]:
# streamlit run stream_test.py
import yfinance as yf
import streamlit as st
import datetime
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import requests_cache
session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'my-program/1.0'

def make_plotly(Company_tiker,date_of_buy='2021-04-28'):
    tickerSymbol = Company_tiker.upper()

    now_now = datetime.datetime.today()

    now_data = now_now.strftime('%Y-%m-%d')

    start_t = datetime.datetime.strptime(date_of_buy,"%Y-%m-%d")
    

    tickerData = yf.Ticker(tickerSymbol)

    tickerDf = tickerData.history(period='1d', start=start_t, end=now_data)

    return tickerDf

In [3]:
def now_price(company_tiker):
    a = make_plotly(company_tiker)
    a['Datetime'] = pd.to_datetime(a.index, format="%m-%d-%Y")
    info_about_last = a[a.Datetime == a.Datetime.max()]
    info_about_last.Close
    return float(info_about_last.Close)

In [60]:
def make_candle(df):
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                                     open=df['Open'], high=df['High'],
                                     low=df['Low'], close=df['Close'])
                      ])
    return fig

In [8]:
def make_dict_from_company(basic_porfolio_dox: 'excel document'):
    dict_companies_data = {}
    for index,row in account.iterrows():
        dict_companies_data[row['tiker']] = make_plotly(row['tiker'],str(row['date_purchase'].strftime('%Y-%m-%d')))
        dict_companies_data[row['tiker']]['date'] = dict_companies_data[row['tiker']].index
    return dict_companies_data

In [41]:
from functools import reduce
def count_mean_measure_all_sec(dict_companies_data):
    ac = pd.DataFrame()
    frames_number = len(dict_companies_data)
    stock_measurets = ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
    for measure in stock_measurets:
        ac[measure] = reduce(
            lambda x,y: dict_companies_data[x][measure]+dict_companies_data[y][measure] ,
        dict_companies_data.keys()
        )/frames_number
    return ac

In [35]:
def porfolio_result(account=account):
    dictionary_info = {}
    dictionary_info['value_in_usd'] = round(account.result.sum())
    dictionary_info['value_in_rub'] =round(account.result.sum())*73
    dictionary_info['buy_sum_total'] = account['sum'].sum()
    dictionary_info['percent'] = dictionary_info['value_in_usd']/dictionary_info['buy_sum_total']
    dictionary_info['percent_display'] = f'{round(dictionary_info["percent"]*100,2)}%'

    return dictionary_info

In [37]:
account = pd.read_excel('my_account.xlsx')
account.date_purchase = pd.to_datetime(account.date_purchase, format="%m-%d-%Y")

In [9]:
dict_companies_data = make_dict_from_company(account)

In [10]:
len(dict_companies_data)

2

In [38]:
account['sum'] = account['price_buy'] * account['lots_buy']

In [39]:
account['now_price'] = list(map(now_price,account.tiker))
account['now_sum'] = round(account['now_price']*account['lots_buy'],0)
account['result'] = account['now_sum'] - account['sum']
account['percentage'] = account['result']/account['sum']
print(round(((account.result.sum()*73)/(account['sum'].sum()*73))*100,2))
account

-17.34


,tiker,price_buy,lots_buy,date_purchase,sum,now_price,now_sum,result,percentage
0,STRA,93,350,2021-04-28,32550,57.650002,20178.0,-12372.0,-0.380092
1,TNDM,94,110,2021-04-28,10340,138.880005,15277.0,4937.0,0.477466


In [40]:
porfolio_result()

{'value_in_usd': -7435,
 'value_in_rub': -542755,
 'buy_sum_total': 42890,
 'percent': -0.17335043133597575,
 'percent_display': '-17.34%'}